In [2]:
import numpy as np
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm  # 폰트 관리

# 파일불러오기
file_path1_4 = './data/tpss_bcycl_od_statnhm_20240104.csv'
file_path1_5 = './data/tpss_bcycl_od_statnhm_20240105.csv'
file_path1_6 = './data/tpss_bcycl_od_statnhm_20240106.csv'

df_rent1_4 = pd.read_csv(file_path1_4,encoding='cp949')
df_rent1_5 = pd.read_csv(file_path1_5,encoding='cp949')
df_rent1_6 = pd.read_csv(file_path1_6,encoding='cp949')

# 전처리

# 날짜별 데이터 합치기
df_rent1 = pd.concat([df_rent1_4,df_rent1_5,df_rent1_6],axis=0)
# 데이터가 많아서 streamlit share에서 에러나서 일단 파일 하나로만 테스트 진행
# df_rent1 = df_rent1_4

# 기준날짜 : csv에서 불러오면 dataframe에서는 int64로 들어오기 때문에 datetime으로 변환
df_rent1['기준_날짜'] = df_rent1['기준_날짜'].astype('str')
# df_rent1['기준_날짜'] = pd.to_datetime(df_rent1['기준_날짜'], format='%Y-%m-%d')

# 종료_대여소명 : NaN --> X
df_rent1['종료_대여소명'].fillna('X',inplace=True)

# Feature Engineering
# df_featured_rent : '시작_대여소_동명', '종료_대여소_동명' 추출
df_featured_rent = df_rent1
df_featured_rent['시작_대여소_동명'] = df_rent1['시작_대여소명'].apply(lambda x:x[:x.find('동')+1])
df_featured_rent['종료_대여소_동명'] = df_rent1['종료_대여소명'].apply(lambda x:x[:x.find('동')+1])



In [8]:
cond = ((df_featured_rent['시작_대여소_동명']=='') | (df_featured_rent['종료_대여소_동명']==''))
df_featured_rent.loc[cond,:].index
df_featured_rent.drop(df_featured_rent.loc[cond,:].index, axis=0, inplace=True)
# len(cond.index)

In [9]:
# cond=df_rent1.groupby('시작_대여소_ID').sum()
cond = ((df_featured_rent['시작_대여소_동명']=='') | (df_featured_rent['종료_대여소_동명']==''))
df_featured_rent.loc[cond,:]

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리,시작_대여소_동명,종료_대여소_동명


In [19]:
cond1 = ((df_rent1['시작_대여소명'].str.contains('보라매동_002_1')) & (df_rent1['종료_대여소명'].str.contains('가산동_003_4')))
cond2 = (df_rent1['집계_기준']=='출발시간')
cond = (cond1 & cond2)
df_rent1.loc[cond,:].sort_values(by=['종료_대여소명']).groupby('시작_대여소_ID').sum()

,기준_날짜,집계_기준,기준_시간대,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리
시작_대여소_ID,,,,,,,,,
ST-1268,20240105,출발시간,1920,보라매동_002_1,ST-1938,가산동_003_4,1,27,4559


In [79]:
cond1 = ((df_rent1['시작_대여소명']=='보라매동_002_1') & (df_rent1['종료_대여소명']=='가산동_003_4'))
cond2 = (df_rent1['집계_기준']=='출발시간')
cond = (cond1 & cond2)

# df_rent1.loc[cond,['기준_날짜','시작_대여소_ID','종료_대여소_ID','전체_건수','전체_이용_분','전체_이용_거리']] \
#         .sort_values(by=['기준_날짜','시작_대여소_ID','종료_대여소_ID']) \
#         .groupby('기준_날짜','시작_대여소_ID','종료_대여소_ID')['전체_건수','전체_이용_분','전체_이용_거리'].sum()
df_rent1.groupby(['기준_날짜','시작_대여소_ID','종료_대여소_ID']).agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})

C:\Users\human\AppData\Local\Temp\ipykernel_13140\2797632983.py:8: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_rent1.groupby(['기준_날짜','시작_대여소_ID','종료_대여소_ID']).agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})
C:\Users\human\AppData\Local\Temp\ipykernel_13140\2797632983.py:8: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_rent1.groupby(['기준_날짜','시작_대여소_ID','종료_대여소_ID']).agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})


전체_건수  전체_이용_분  전체_이용_거리
기준_날짜    시작_대여소_ID 종료_대여소_ID                          
20240104 ST-10     ST-10          8      200      4222
                   ST-1190        2       18      4278
                   ST-1342        2       96     16776
                   ST-16          4       16      2588
                   ST-20          4       18      2954
...                             ...      ...       ...
20240106 ST-997    ST-530         2       18      3242
                   ST-993         2      202      9060
                   ST-994         6       34      5670
                   ST-996         2       36      4036
                   ST-997        28     1094     35822

[149628 rows x 3 columns]

In [26]:
cond1 = ((df_rent1['시작_대여소명']=='보라매동_002_1'))
cond2 = (df_rent1['집계_기준']=='도착시간')
cond = (cond1 & cond2)
df_rent1.loc[cond1,:].sort_values(by=['종료_대여소명','기준_시간대'])

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리
152444,20240105,출발시간,1920,ST-1268,보라매동_002_1,ST-1938,가산동_003_4,1,27,4559
157758,20240105,도착시간,1945,ST-1268,보라매동_002_1,ST-1938,가산동_003_4,1,27,4559
114427,20240105,출발시간,1710,ST-1268,보라매동_002_1,ST-679,개봉3동_034_1,1,49,12490
126826,20240105,도착시간,1755,ST-1268,보라매동_002_1,ST-679,개봉3동_034_1,1,49,12490
93550,20240105,출발시간,1510,ST-1268,보라매동_002_1,ST-676,고척2동_035_1,1,49,10153
...,...,...,...,...,...,...,...,...,...,...
35131,20240106,도착시간,1105,ST-1268,보라매동_002_1,ST-715,청룡동_034_2,1,14,2220
22898,20240106,출발시간,915,ST-1268,보라매동_002_1,X,NaN,1,10,3070
31622,20240106,도착시간,1035,ST-1268,보라매동_002_1,X,NaN,1,10,3070
136381,20240104,출발시간,1900,ST-1268,보라매동_002_1,X,NaN,1,17,3911


In [29]:
cond = df_rent1['종료_대여소명'].isnull()
df_rent1.loc[cond,:]

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리
924,20240104,출발시간,20,ST-3214,방이2동_007_1,X,NaN,1,100,1671
1096,20240104,출발시간,25,ST-1086,오륜동_001_1,X,NaN,1,56,893
1175,20240104,출발시간,25,ST-4,서교동_017_1,X,NaN,1,5,3881
1320,20240104,출발시간,30,ST-1421,암사3동_004_1,X,NaN,1,5,669
1597,20240104,출발시간,35,ST-2024,가양1동_025_2,X,NaN,1,60,3075
...,...,...,...,...,...,...,...,...,...,...
110734,20240106,도착시간,2235,ST-3072,여의동_005_4,X,NaN,1,110,2212
111004,20240106,도착시간,2245,ST-2477,방화1동_080_1,X,NaN,1,30,4692
111513,20240106,도착시간,2305,ST-1689,가양1동_039_3,X,NaN,1,60,3605
112121,20240106,출발시간,2330,ST-2395,신정6동_019_2,X,NaN,1,10,0


In [84]:
cond = df_rent1['시작_대여소명'].str.contains('동')
df_rent1.loc[~cond,:]

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리
2672,20240104,출발시간,100,ST-1289,_39,ST-2698,상계9동_007_1,1,11,2320
3456,20240104,도착시간,115,ST-1289,_39,ST-2698,상계9동_007_1,1,11,2320
5170,20240104,출발시간,205,ST-3190,_67,ST-930,거여2동_001_1,1,11,2000
5703,20240104,도착시간,220,ST-3190,_67,ST-930,거여2동_001_1,1,11,2000
10554,20240104,출발시간,600,ST-1289,_39,ST-612,상계1동_002_1,1,8,990
...,...,...,...,...,...,...,...,...,...,...
98758,20240106,도착시간,1815,ST-3191,_68,ST-2572,위례동_003_1,1,12,1420
103165,20240106,출발시간,1900,ST-3190,_67,ST-2936,마천1동_019_1,1,67,2790
107059,20240106,도착시간,2010,ST-3190,_67,ST-2936,마천1동_019_1,1,67,2790
109333,20240106,출발시간,2150,ST-3191,_68,ST-3191,_68,1,3,21


In [102]:
cond = df_rent1['종료_대여소명'].isnull()
# df_rent1.loc[cond,:]
df_rent1['종료_대여소명'].fillna('X',inplace=True)
df_rent1.loc[cond,:]

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리,시작_대여소_동명


In [103]:
df_rent1['시작_대여소_동명'] = df_rent1['시작_대여소명'].apply(lambda x:x[:x.find('동')+1])
df_rent1['종료_대여소_동명'] = df_rent1['종료_대여소명'].apply(lambda x:x[:x.find('동')+1])
df_rent1

,기준_날짜,집계_기준,기준_시간대,시작_대여소_ID,시작_대여소명,종료_대여소_ID,종료_대여소명,전체_건수,전체_이용_분,전체_이용_거리,시작_대여소_동명,종료_대여소_동명
0,20240104,출발시간,0,ST-1014,신정3동_101_1,ST-676,고척2동_035_1,1,6,2320,신정3동,고척2동
1,20240104,출발시간,0,ST-1017,목5동_059_1,ST-310,목4동_047_1,1,6,909,목5동,목4동
2,20240104,출발시간,0,ST-1055,강일동_001_1,ST-2744,강일동_001_5,1,93,12860,강일동,강일동
3,20240104,출발시간,0,ST-1082,장지동_019_1,ST-2599,장지동_012_3,1,5,910,장지동,장지동
4,20240104,출발시간,0,ST-1092,풍납2동_026_1,ST-502,성내1동_007_1,1,5,1063,풍납2동,성내1동
...,...,...,...,...,...,...,...,...,...,...,...,...
112737,20240106,도착시간,2355,ST-884,중앙동_009_1,ST-1548,은천동_065_1,1,6,960,중앙동,은천동
112738,20240106,도착시간,2355,ST-913,월계3동_039_1,ST-2708,월계2동_025_1,1,9,1569,월계3동,월계2동
112739,20240106,도착시간,2355,ST-919,고척1동_006_1,ST-1227,개봉1동_050_1,1,6,353,고척1동,개봉1동
112740,20240106,도착시간,2355,ST-93,상암동_026_4,ST-43,신촌동_030_2,1,37,4579,상암동,신촌동


In [15]:
df_featured_rent.groupby(['시작_대여소_동명'])\
             .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\
             .sort_values(by=['전체_건수','전체_이용_분','전체_이용_거리'],ascending=False)

C:\Users\human\AppData\Local\Temp\ipykernel_13868\1226884225.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\
C:\Users\human\AppData\Local\Temp\ipykernel_13868\1226884225.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\


,전체_건수,전체_이용_분,전체_이용_거리
시작_대여소_동명,,,
가양1동,21593,247302,29595648
여의동,9392,205970,22035727
발산1동,8703,99372,11312402
방화1동,7331,80467,8683627
목1동,7235,114058,11636436
...,...,...,...
홍제1동,32,844,101336
정릉4동,32,542,63502
반포본동,24,238,33546


In [16]:
df_for_count = df_featured_rent.groupby(['시작_대여소_동명'])\
                .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\
                .sort_values(by=['전체_건수','전체_이용_분','전체_이용_거리'],ascending=False)

C:\Users\human\AppData\Local\Temp\ipykernel_13868\183667873.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\
C:\Users\human\AppData\Local\Temp\ipykernel_13868\183667873.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\


In [18]:
df_for_count.shape[0]

412

In [37]:
df_featured_rent.groupby(['시작_대여소_동명'])\
        .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\
        .sort_values(by=['전체_건수','전체_이용_분','전체_이용_거리'],ascending=False).shape[0]

C:\Users\human\AppData\Local\Temp\ipykernel_13868\815253628.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\
C:\Users\human\AppData\Local\Temp\ipykernel_13868\815253628.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .agg({'전체_건수':sum,'전체_이용_분':sum,'전체_이용_거리':sum})\


412

In [33]:
df.iloc[:10,:]

,전체_건수,전체_이용_분,전체_이용_거리
시작_대여소_동명,,,
가양1동,21593,247302,29595648
여의동,9392,205970,22035727
발산1동,8703,99372,11312402
방화1동,7331,80467,8683627
목1동,7235,114058,11636436
당산2동,5704,88947,8838633
종로1·2·3·4가동,5680,111869,10302066
공항동,5454,59463,6897313
목5동,5124,80074,8511178
